# Asthma-Like Illness Emergency Department Presentations (annual) | Processing

The main tasks completed to clean and preprocess this dataset were:

**Data Manipulation**
1. Rename columns.
2. Reformat 'financial year' values from XX/YY to XXXX/YYYY
3. Remove 'LHD' from LHD name values.
4. Remove 'All' data (Representing a state-wide average).
5. Remove columns holding Confidence Interval data.
6. Remove rows holding 'Persons' data in the sex column (Representing a genderless rate per 100,000).

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [3]:
import pandas as pd

## Load Dataset

In [4]:
# File path.
file_path = 'data-raw.csv'

# Read the file.
df = pd.read_csv(file_path)

df

,Sex,LHD,Period,"Rate per 100,000 population",LL 95% CI,UL 95% CI
0,Males,Sydney LHD,14/15,283.1,263.1,304.1
1,Males,Sydney LHD,15/16,287.6,267.5,308.7
2,Males,Sydney LHD,16/17,263.7,244.7,283.8
3,Males,Sydney LHD,17/18,229.5,211.8,248.4
4,Males,Sydney LHD,18/19,244.4,226.1,263.7
...,...,...,...,...,...,...
847,NaN,NaN,NaN,NaN,NaN,NaN
848,NaN,NaN,NaN,NaN,NaN,NaN
849,NaN,NaN,NaN,NaN,NaN,NaN
850,NaN,NaN,NaN,NaN,NaN,NaN


## Data Manipulation

Rename columns to match Air Quality data set.

In [3]:
# Rename columns.
df = df.rename(columns={
    'LHD': 'lhd',
    'Period': 'financial year'
})

# Set column names to lower case.
df.columns = df.columns.str.lower()

Mark Year data in 'financial year' column more verbose.

In [4]:
# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

Remove ' LHD' for Local Health District values.

In [5]:
# Remove ' LHD' from the 'lhd' column.
df['lhd'] = df['lhd'].str.replace(' LHD', '')

Remove rows representing state-wide aggregated data.

In [6]:
# Remove rows with NaN in the 'lhd' column.
df = df.dropna(subset=['lhd'])

# Remove rows with 'All' in the 'lhd' column.
df = df[~df['lhd'].str.contains('All')]

Remove columns holding Confidence Interval data.

In [7]:
# Drop columns with '% ci' in the header
df = df.loc[:, ~df.columns.str.contains('% ci')]

Remove rows holding 'Persons' data in the sex column.

In [8]:
# Drop rows with 'Persons' in the 'sex' column.
df = df[~df['sex'].str.contains('Persons')]

## Output Processed Dataset

In [9]:
# File path.
file_path_output = 'data-processed.csv'

# Save the file.
df.to_csv(file_path_output, index=False)

## View Dataset

In [10]:
df

,sex,lhd,financial year,"rate per 100,000 population"
0,Males,Sydney,2014/2015,283.1
1,Males,Sydney,2015/2016,287.6
2,Males,Sydney,2016/2017,263.7
3,Males,Sydney,2017/2018,229.5
4,Males,Sydney,2018/2019,244.4
...,...,...,...,...
274,Females,Far West,2018/2019,884.7
275,Females,Far West,2019/2020,692.3
276,Females,Far West,2020/2021,592.1
277,Females,Far West,2021/2022,689.7


## Alternative Approach
Composite primary key becomes only 'lhd' and 'date'.

Tracks 'Male rate per 100,000 population' and 'Female rate per 100,000 population' on the same row.

### Reconfigure Table

In [ ]:
# Pivot the dataframe to have 'sex' as columns
df_alt = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df_alt.columns.name = None
df_alt = df_alt.rename(columns={'Males': 'Male rate per 100,000 population', 'Females': 'Female rate per 100,000 population'})

df_alt

,date,lhd,"Female rate per 100,000 population","Male rate per 100,000 population"
0,2014-07,Central Coast,29.8,21.8
1,2014-07,Hunter New England,45.3,32.3
2,2014-07,Illawarra Shoalhaven,42.1,26.7
3,2014-07,Mid North Coast,58.4,35.5
4,2014-07,Murrumbidgee,46.2,51.4
...,...,...,...,...
1507,2023-06,South Western Sydney,18.9,18.4
1508,2023-06,Southern NSW,29.7,40.7
1509,2023-06,Sydney,16.9,14.4
1510,2023-06,Western NSW,55.6,38.1


### Output Alternative Processed Dataset

In [ ]:
# File path.
file_path_output_alt = 'data-processed-alt.csv'

# Save the file.
df_alt.to_csv(file_path_output_alt, index=False)